In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/nyc_311_requests.csv")

/tmp/ipykernel_79692/473709468.py:1: DtypeWarning: Columns (18,20,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/nyc_311_requests.csv")


In [3]:
df.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [6]:
kept_columns = [
    'Unique Key',
    'Created Date',
    'Closed Date',
    'Agency',
    'Agency Name',
    'Complaint Type',
    'Descriptor',
    'Location Type',
    'Incident Zip',
    'Borough',
    'Status',
    'Latitude',
    'Longitude']

clean_311 = df[kept_columns].copy()

In [7]:
clean_311.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,Status,Latitude,Longitude
0,66998447,11/30/2025 08:11:09 PM,11/30/2025 09:57:45 PM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10461.0,BRONX,Closed,40.839383,-73.845302
1,66999600,11/30/2025 08:11:04 PM,11/30/2025 08:48:31 PM,NYPD,New York City Police Department,Non-Emergency Police Matter,Trespassing,Residential Building/House,11231.0,BROOKLYN,Closed,40.674069,-74.002188
2,66997399,11/30/2025 08:10:49 PM,12/01/2025 12:00:08 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466.0,BRONX,Closed,40.891872,-73.860168
3,67000529,11/30/2025 08:10:47 PM,12/02/2025 03:55:10 PM,DCWP,Department of Consumer and Worker Protection,Consumer Complaint,"Bodega, Deli, or Convenience Store",Business,11237.0,BROOKLYN,Closed,40.703016,-73.925736
4,66998834,11/30/2025 08:10:19 PM,12/03/2025 08:40:54 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10019.0,MANHATTAN,Closed,40.765993,-73.985206


In [10]:
clean_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295008 entries, 0 to 295007
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unique Key      295008 non-null  int64  
 1   Created Date    295008 non-null  object 
 2   Closed Date     268244 non-null  object 
 3   Agency          295008 non-null  object 
 4   Agency Name     295008 non-null  object 
 5   Complaint Type  295008 non-null  object 
 6   Descriptor      290008 non-null  object 
 7   Location Type   264942 non-null  object 
 8   Incident Zip    292631 non-null  float64
 9   Borough         295008 non-null  object 
 10  Status          295008 non-null  object 
 11  Latitude        292165 non-null  float64
 12  Longitude       292165 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 29.3+ MB


In [8]:
clean_311.isnull().sum()

Unique Key            0
Created Date          0
Closed Date       26764
Agency                0
Agency Name           0
Complaint Type        0
Descriptor         5000
Location Type     30066
Incident Zip       2377
Borough               0
Status                0
Latitude           2843
Longitude          2843
dtype: int64

Checking for duplicates and filling in unknowns

In [9]:
clean_311.duplicated().sum()

np.int64(0)

In [12]:
clean_311['Incident Zip'] = clean_311['Incident Zip'].fillna('Unknown')

Extracting Datetime

In [13]:
clean_311['Created Date'] = pd.to_datetime(clean_311['Created Date'],errors='coerce')
clean_311['Closed Date'] = pd.to_datetime(clean_311['Closed Date'],errors='coerce')

/tmp/ipykernel_79692/972804068.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_311['Created Date'] = pd.to_datetime(clean_311['Created Date'],errors='coerce')
/tmp/ipykernel_79692/972804068.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_311['Closed Date'] = pd.to_datetime(clean_311['Closed Date'],errors='coerce')


Feature Engineering

In [15]:
clean_311['Year'] = clean_311['Created Date'].dt.year
clean_311['Month'] = clean_311['Created Date'].dt.month
clean_311['Week Day'] = clean_311['Created Date'].dt.day_name()
clean_311['Hour'] = clean_311['Created Date'].dt.hour
clean_311['Resolution_time Hours'] = (clean_311['Closed Date'] - clean_311['Created Date']).dt.total_seconds() / 3600
clean_311['Resolution_time Days'] = clean_311['Resolution_time Hours'] / 24

In [16]:
clean_311.to_csv("../data/clean_nyc_311_requests.csv", index=False)